# Setup 

## Libraries 

In [35]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.patches import Ellipse, Polygon 
from matplotlib.animation import FuncAnimation 
import plotly.graph_objects as go 

## Folder Paths 

In [36]:
import os 

folder_path = os.environ.get("NFL_DATA_PATH") 
results_path = os.environ.get("NFL_RESULTS_PATH") 

# Animation Calculations 

## process_keyframes 

In [37]:
# function to take a dictionary of keyframes and create a dataframe from it 
def process_keyframes(keyframes): 

    # initial dataframe 
    dfk = pd.DataFrame() 

    # loop through each frame and create a dataframe for it 
    for frame in keyframes:
        dfk = pd.concat([dfk, pd.DataFrame({
            "TIME": frame['time'], 
            "PARAM": frame['values'].keys(), 
            "VALUE": frame['values'].values() 
        })]) 
    
    # reset the index 
    dfk = dfk.reset_index(drop = True) 

    return dfk 

# some sample keyframes 
keyframes = [
    {
        "time": 0, 
        "values": {
            "xc": 30, 
            "yc": 90 
        }
    }, 
    {
        "time": 1, 
        "values": {
            "yc": 100 
        }
    }
] 

# showcase the function 
dfk = process_keyframes(keyframes) 
dfk 

,TIME,PARAM,VALUE
0,0,xc,30
1,0,yc,90
2,1,yc,100


## params_at_time 

In [38]:
# function to calculate parameter values at a given time 
def params_at_time(t, dfk): 

    # sort by time 
    dfk = dfk.sort_values(["TIME"]) 

    # get the parameter values before 
    df_before = (
        dfk[dfk["TIME"] <= t].groupby("PARAM").last() 
        .rename(columns = {"TIME": "TIME_BEFORE", "VALUE": "VALUE_BEFORE"}) 
    ) 

    # get the parameter values after
    df_after = (
        dfk[dfk["TIME"] > t].groupby("PARAM").first() 
        .rename(columns = {"TIME": "TIME_AFTER", "VALUE": "VALUE_AFTER"})
    ) 

    # merge the two and add the time 
    df_merged = df_before.merge(df_after, on = ["PARAM"], how = "outer").reset_index()  
    df_merged["TIME"] = t 

    # calculate the interpolated value
    def interp_value(row): 

        # if either is na, return the other 
        if pd.isna(row["VALUE_BEFORE"]): 
            return row["VALUE_AFTER"] 
        elif pd.isna(row["VALUE_AFTER"]): 
            return row["VALUE_BEFORE"] 

        # otherwise, do a linear interpolation 
        else: 
            t0 = row["TIME_BEFORE"] 
            t1 = row["TIME_AFTER"] 
            v0 = row["VALUE_BEFORE"] 
            v1 = row["VALUE_AFTER"] 
            return v0 + (v1 - v0) * ((t - t0) / (t1 - t0)) 
    
    # apply the interpolation function 
    df_merged["VALUE"] = df_merged.apply(interp_value, axis = 1) 

    # transform to a dictionary 
    pdict = {row["PARAM"]: row["VALUE"] for _, row in df_merged.iterrows()}

    return pdict 

# showcase the function 
pdict = params_at_time(3, dfk) 
pdict 

{'xc': 30, 'yc': 100}

## keyframes_to_params 

In [39]:
# function to transform keyframes into parameter values at a given time 
def keyframes_to_params(t, keyframes, defaults):

    # if no keyframes, just take defaults 
    if len(keyframes) == 0:
        pdict = defaults 

    # otherwise, process keyframes and fill in defaults 
    else:
        dfk = process_keyframes(keyframes) 
        pdict = params_at_time(t, dfk) 

        # fill in default parameters 
        for key, value in defaults.items(): 
            if key not in pdict: 
                pdict[key] = value 
    
    return pdict 

# showcase the function 
keyframes_to_params(
    t = 3, 
    keyframes = keyframes, 
    defaults = {
        "xc": 0, 
        "yc": 0, 
        "alpha": 1 
    }
) 

{'xc': 30, 'yc': 100, 'alpha': 1}

# Animation Objects 

## PlayAnimation 

In [40]:
# class for each play animation 
class PlayAnimation:

    def __init__(self, ystart = 0, yend = 120):
        self.ystart = ystart 
        self.yend = yend 
        self.pixels_width = 1920 
        self.pixels_height = int(self.pixels_width * ((yend - ystart) / 53.3)) 
        self.dpi = 120 
        self.objects = {}  
    
    # create a figure object based on the sizing 
    def create_figure(self):
        return plt.Figure(
            figsize = (
                self.pixels_width / self.dpi, 
                self.pixels_height / self.dpi
            ), 
            dpi = self.dpi 
        ) 

    # create an axis object based on the sizing 
    def create_axis(self, fig, margin = 5):
        return fig.add_axes([
            margin / self.pixels_width, 
            margin / self.pixels_height, 
            1 - (2 * margin / self.pixels_width), 
            1 - (2 * margin / self.pixels_height)
        ]) 
    
    # add an object to the play 
    def add_object(self, obj):
        self.objects[obj.id] = obj 
    
    # add keyframes for the players 
    def add_keyframes(self, keyframes):
        for k, v in keyframes.items():
            if k in self.objects:
                self.objects[k].keyframes += v 

    # draw the play at the current time 
    def draw_play(self, fig, t = 0): 

        # create a new axis 
        ax = self.create_axis(fig) 

        # draw out the objects 
        for obj in self.objects.values():
            obj.draw(ax, t = t) 

        # Set limits and hide axes 
        ax.set_xlim(0, 53.3)
        ax.set_ylim(self.ystart, self.yend) 
        ax.axis('off') 
    
    # save an image of the play at time t 
    def save_image(self, t = 0, save_file = "output_test.png"):

        # draw the play 
        fig = self.create_figure() 
        self.draw_play(fig, t = t) 
        
        # save the figure output for testing 
        fig.savefig(save_file, dpi = self.dpi, pad_inches = 0)

    # create an animation of the play over time 
    def create_animation(self, end_time, fps = 12, save_file = "output_animation.mp4"):
        
        # calculate the times at each animation frame 
        tlist = np.linspace(0, end_time, int(np.max([np.round(end_time * fps), 1]))) 

        # create the figure and axis 
        fig = self.create_figure() 
        ax = self.create_axis(fig) 

        # create the animation function 
        def animate(i):
            if i % 20 == 1:
                print(f"  Creating frame {i} / {len(tlist)}...") 

            # clear out the previous frame 
            fig.clf() 
                
            # draw the play at time tlist[i] 
            self.draw_play(fig, t = tlist[i]) 
        
        # compile and save the animation 
        ani = FuncAnimation(fig, animate, frames = len(tlist) )
        ani.save(save_file, writer = 'ffmpeg', fps = fps) 

## Field 

### draw_field 

In [41]:
# function to draw the field background 
def draw_field(ax, alpha = 1): 

    # Set the background color to green
    ax.add_patch(plt.Rectangle(
        xy = (0, 0), 
        height = 120, 
        width = 53.3, 
        color = [0.32, 0.56, 0.23, alpha], 
        zorder = 0 
    )) 

    # add rectangles for the endzones 
    for i in [0, 110]:
        ax.add_patch(plt.Rectangle(
            xy = (0, i), 
            height = 10, 
            width = 53.3, 
            color = [0.22, 0.38, 0.16, alpha], 
            zorder = 1  
        )) 

    # draw a line for every 5 yards 
    for i in range(0, 21):
        ax.plot(
            [0, 53.3], 
            [10 + (i * 5)] * 2, 
            color = 'white', 
            linewidth = [4, 2][i % 2], 
            alpha = alpha, 
            zorder = 2 
        ) 

    # add the hash marks 
    for x in [20.65, 32.65]: # left and right hash marks 
        for y in range(0, 20):
            yc = (y * 5) + 10 
            ax.plot(
                [x] * 2, 
                [yc - 0.5, yc + 0.5], 
                color = 'white', 
                linewidth = 2, 
                alpha = alpha, 
                zorder = 2 
            ) 

    # add markers for each yard line 
    for x in [0.5, 20.65, 32.65, 52.8]:
        for yl in range(0, 100):
            ax.plot(
                [x - 0.5, x + 0.5], 
                [yl + 10] * 2, 
                color = 'white', 
                linewidth = 2, 
                alpha = alpha, 
                zorder = 2 
            ) 

    # add the numbers 
    for i, txt in enumerate(["1 0", "2 0", "3 0", "4 0", "5 0", "4 0", "3 0", "2 0", "1 0"]):
        for x in [8, 53.3 - 8]:
            ax.text(
                x = x, 
                y = (i * 10) + 20, 
                s = txt, 
                color = 'white', 
                fontsize = 44, 
                fontweight = 'bold', 
                horizontalalignment = 'center', 
                verticalalignment = 'center', 
                rotation = -90 if x < 26.65 else 90, 
                alpha = 0.65 * alpha, 
                zorder = 2 
            ) 

### FieldObject 

In [42]:
# class to represent the field background 
class FieldObject:

    def __init__(self, id = "field"):
        self.id = id 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "alpha": 1 
        }

    # draw the field background
    def draw(self, ax, t = 0):

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the field 
        draw_field(ax, alpha = pdict["alpha"]) 

# create the play animation object 
pa = PlayAnimation(ystart = 85, yend = 120) 

# add the field object 
pa.add_object(FieldObject()) 

# test it out 
pa.save_image() 
plt.close() 

## Players 

### rotate_coords 

In [43]:
# function to rotate coordinates based on a given angle 
def rotate_coords(center, shift, angle_deg):
    angle_rad = np.radians(-angle_deg)
    return [
        (shift[0] * np.cos(angle_rad)) - (shift[1] * np.sin(angle_rad)) + center[0], 
        (shift[0] * np.sin(angle_rad)) + (shift[1] * np.cos(angle_rad)) + center[1] 
    ] 

# showcase the function 
rotate_coords(center = [10, 10], shift = [1, 0], angle_deg = 80) 

[np.float64(10.17364817766693), np.float64(9.015192246987793)]

### draw_player 

In [44]:
# function to draw out a player 
def draw_player(ax, x, y, angle = 0, pcolor = "red", alpha = 1):

    # draw the body ellipse 
    ax.add_patch(Ellipse(
        xy = (x, y), 
        width = 1.5, 
        height = 0.75, 
        angle = -angle, 
        facecolor = pcolor, 
        edgecolor = "black", 
        linewidth = 2,
        zorder = 25, 
        alpha = alpha 
    )) 

    # draw the head ellipse 
    ax.add_patch(Ellipse(
        xy = (x, y), 
        width = 0.5, 
        height = 0.5, 
        angle = -angle, 
        facecolor = pcolor, 
        edgecolor = "black", 
        linewidth = 2,
        zorder = 26, 
        alpha = alpha 
    )) 

    # trapezoid dimensions  
    dw1 = 0.25 
    dw2 = 1   
    dh = 0.75  

    # calculate rotated corners 
    corners = [[-dw1, 0], [-dw2, dh], [dw2, dh], [dw1, 0]] 
    rotated_corners = [rotate_coords(center = [x, y], shift = corner, angle_deg = angle) for corner in corners] 
    # angle_rad = np.radians(-angle) 
    # corners = [[-dw1, 0], [-dw2, dh], [dw2, dh], [dw1, 0]]
    # rotated_corners = [] 
    # for cx, cy in corners:
    #     new_x = cx * np.cos(angle_rad) - cy * np.sin(angle_rad) + x
    #     new_y = cx * np.sin(angle_rad) + cy * np.cos(angle_rad) + y
    #     rotated_corners.append([new_x, new_y])

    # draw the polygon 
    ax.add_patch(Polygon(
        rotated_corners, 
        facecolor = "yellow", 
        edgecolor = None, 
        alpha = 0.65 * alpha, 
        zorder = 24 
    )) 

### PlayerObject 

In [45]:
# class object to define each player 
class PlayerObject:

    def __init__(self, id, pcolor = "red"): 

        # initial parameters 
        self.id = id 
        self.pcolor = pcolor 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "xc": 0, 
            "yc": 0, 
            "angle": 0, 
            "alpha": 1 
        } 

    # draw the player at time t 
    def draw(self, ax, t = 0): 

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the player 
        draw_player(
            ax = ax, 
            x = pdict["xc"], 
            y = pdict["yc"], 
            angle = pdict["angle"], 
            pcolor = self.pcolor, 
            alpha = pdict["alpha"] 
        )

# create the play animation object with the field 
pa = PlayAnimation(ystart = 85, yend = 120) 
pa.add_object(FieldObject()) 

# add some players 
pa.add_object(PlayerObject(id = "1", pcolor = "blue")) 
pa.add_object(PlayerObject(id = "2", pcolor = "red")) 

# add keyframes for the players 
pa.add_keyframes({
    "1": [
        {
            "time": 0, 
            "values": {
                "xc": 30, 
                "yc": 90, 
                "angle": 0 
            }
        } 
    ],
    "2": [
        {
            "time": 0, 
            "values": {
                "xc": 35, 
                "yc": 94, 
                "angle": 240 
            }
        }
    ]
}) 

# draw and save the play 
pa.save_image() 
plt.close() 

## Routes 

### create_routes_df 

In [46]:
# function to create a dataframe from the given routes 
def create_routes_df(routes): 

    # initial dataframe 
    dfr = pd.DataFrame() 

    # loop through each route 
    for i, route in enumerate(routes):

        # create a dataframe for the route 
        dfr = pd.concat([dfr, pd.DataFrame({
            "TIME": [((frame - 1) / route["fps"]) + route["start_time"] for frame in route["path"]["frame_id"]],
            "XC": route["path"]["xc"], 
            "YC": route["path"]["yc"] 
        })], ignore_index=True) 
    
    # reset the index 
    dfr = dfr.sort_values("TIME").reset_index(drop = True) 

    # get the previous values 
    dfr["PREV_XC"] = dfr["XC"].shift(1)
    dfr["PREV_YC"] = dfr["YC"].shift(1)
    dfr["PREV_TIME"] = dfr["TIME"].shift(1) 

    return dfr 

# sample routes data 
routes = [
    {
        "start_time": 0, 
        "fps": 10, 
        "path": {
            "frame_id": [1, 2, 3, 4], 
            "xc": [1, 2, 3, 4], 
            "yc": [2, 4, 6, 8] 
        }
    }
] 

# test the function 
dfr = create_routes_df(routes) 
dfr 

,TIME,XC,YC,PREV_XC,PREV_YC,PREV_TIME
0,0.0,1,2,NaN,NaN,NaN
1,0.1,2,4,1.0,2.0,0.0
2,0.2,3,6,2.0,4.0,0.1
3,0.3,4,8,3.0,6.0,0.2


### calc_route_line 

In [47]:
# function to calculate a line at the current time 
def calc_route_line(t, dfr): 

    # loop through each row and add to the arrays 
    xlist = [] 
    ylist = [] 
    for i, row in dfr.iterrows():

        # if time is before or at the current time, add the point 
        if row["TIME"] <= t:
            xlist.append(row["XC"]) 
            ylist.append(row["YC"]) 

        # if time is after the current time, interpolate and break 
        elif row["PREV_TIME"] < t and pd.notna(row["PREV_TIME"]):
            t0 = row["PREV_TIME"] 
            t1 = row["TIME"] 
            x0 = row["PREV_XC"] 
            x1 = row["XC"] 
            y0 = row["PREV_YC"] 
            y1 = row["YC"] 
            x_interp = x0 + (x1 - x0) * ((t - t0) / (t1 - t0)) 
            y_interp = y0 + (y1 - y0) * ((t - t0) / (t1 - t0)) 
            xlist.append(x_interp) 
            ylist.append(y_interp) 
            break 
    
    return xlist, ylist

# test the function 
calc_route_line(0.38, dfr)

([np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)],
 [np.float64(2.0), np.float64(4.0), np.float64(6.0), np.float64(8.0)])

### RouteObject 

In [48]:
# class for player routes/paths 
class RouteObject:

    def __init__(self, id, routes = [], rcolor = "blue"): 

        # initial parameters 
        self.id = id 
        self.rcolor = rcolor 
        self.keyframes = [] 
        self.routes = routes

        # default parameters 
        self.defaults = { 
            "alpha": 1 
        } 
    
    # draw the route at time t 
    def draw(self, ax, t = 0): 

        # calulate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # calculate route path at time t 
        dfr = create_routes_df(self.routes) 
        xlist, ylist = calc_route_line(t, dfr) 

        # draw the route
        ax.plot(
            xlist, 
            ylist, 
            linewidth = 4, 
            color = self.rcolor, 
            alpha = self.defaults["alpha"], 
            zorder = 15 
        ) 

# create the play animation with the field 
pa = PlayAnimation(ystart = 85, yend = 120) 
pa.add_object(FieldObject()) 

# add a route object 
pa.add_object(RouteObject(
    id = "route_1", 
    routes = [
        {
            "start_time": 0, 
            "fps": 10, 
            "path": {
                "frame_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                "xc": [20, 21, 22, 23, 24, 24, 24, 23, 22, 21], 
                "yc": [90, 91, 92, 93, 94, 95, 96, 97, 98, 99] 
            }
        }
    ], 
    rcolor = "blue"
)) 

# draw and save the play 
pa.save_image(t = 0.85) 
plt.close() 

## Metrics 

### MetricObject 

In [49]:
# class object to display a metric 
class MetricObject:

    def __init__(self, id, suffix = ""):

        # initial parameters 
        self.id = id 
        self.suffix = suffix 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "xc": 0, 
            "yc": 0, 
            "value": 0, 
            "alpha": 1 
        } 

    # draw the metric at time t 
    def draw(self, ax, t = 0): 

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the metric text 
        ax.text(
            x = pdict["xc"], 
            y = pdict["yc"], 
            s = f"{pdict['value']:.1f}{self.suffix}", 
            color = "black", 
            fontsize = 24, 
            fontweight = 'bold', 
            horizontalalignment = 'center', 
            verticalalignment = 'center', 
            alpha = pdict["alpha"], 
            zorder = 30 
        ) 

## Football 

### draw_football

In [50]:
# function to draw out the football 
def draw_football(ax, x, y, angle = 0, alpha = 1):

    # draw the ellipse 
    ax.add_patch(Ellipse(
        xy = (x, y), 
        width = 0.5, 
        height = 1.0, 
        angle = -angle, 
        facecolor = "brown", 
        edgecolor = "black", 
        linewidth = 2,
        zorder = 32, 
        alpha = alpha 
    )) 

### FootballObject 

In [51]:
# class object to define the football 
class FootballObject:

    def __init__(self, id): 

        # initial parameters 
        self.id = id 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "xc": 0, 
            "yc": 0, 
            "angle": 0, 
            "alpha": 1 
        } 

    # draw the football at time t 
    def draw(self, ax, t = 0): 

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the football 
        draw_football(
            ax = ax, 
            x = pdict["xc"], 
            y = pdict["yc"], 
            angle = pdict["angle"], 
            alpha = pdict["alpha"] 
        )

# create the play animation object with the field 
pa = PlayAnimation(ystart = 85, yend = 120) 
pa.add_object(FieldObject()) 

# add some players 
pa.add_object(FootballObject(id = "ball")) 

# add keyframes for the players 
pa.add_keyframes({
    "ball": [
        {
            "time": 0, 
            "values": {
                "xc": 30, 
                "yc": 100, 
                "angle": 0 
            }
        } 
    ] 
}) 

# draw and save the play 
pa.save_image() 
plt.close() 

## Quick Demo

In [52]:
# # create the play animation with the field 
# pa = PlayAnimation(ystart = 85, yend = 120) 
# pa.add_object(FieldObject()) 

# # add some players 
# pa.add_object(PlayerObject(id = "1", pcolor = "blue")) 
# pa.add_object(PlayerObject(id = "2", pcolor = "red")) 

# # add keyframes for the players 
# pa.add_keyframes({
#     "1": [
#         {
#             "time": 0, 
#             "values": {
#                 "xc": 22, 
#                 "yc": 90, 
#                 "angle": -80 
#             }
#         }, 
#         {
#             "time": 2, 
#             "values": {
#                 "xc": 17, 
#                 "yc": 95, 
#                 "angle": -20 
#             }
#         } 
#     ], 
#     "2": [
#         {
#             "time": 0, 
#             "values": {
#                 "xc": 25, 
#                 "yc": 100, 
#                 "angle": -170 
#             }
#         }, 
#         {
#             "time": 2, 
#             "values": {
#                 "xc": 18, 
#                 "yc": 96, 
#                 "angle": -100 
#             }
#         } 
#     ]
# }) 

# # add a route for the blue player 
# pa.add_object(RouteObject(
#     id = "route_1", 
#     routes = [
#         {
#             "start_time": 0, 
#             "fps": 0.5, 
#             "path": {
#                 "frame_id": [1, 2], 
#                 "xc": [22, 17], 
#                 "yc": [90, 95] 
#             }
#         }
#     ], 
#     rcolor = "blue"
# )) 

# # create and save the animation 
# pa.create_animation(
#     end_time = 2.5, 
#     fps = 12, 
#     save_file = "test_animation.mp4"
# ) 
# plt.close() 

# Add Play Data 

In [53]:
# read in the play metadata
df_meta = pd.read_csv(f"{folder_path}/supplementary_data.csv") 

# showcase the data 
df_meta.head() 

C:\Users\rjfis\AppData\Local\Temp\ipykernel_21536\1083874482.py:2: DtypeWarning:

Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.



,game_id,season,week,game_date,game_time_eastern,home_team_abbr,visitor_team_abbr,play_id,play_description,quarter,...,team_coverage_type,penalty_yards,pre_penalty_yards_gained,yards_gained,expected_points,expected_points_added,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,home_team_win_probability_added,visitor_team_win_probility_added
0,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,3461,(10:46) (Shotgun) J.Goff pass deep left to J.R...,4,...,COVER_2_ZONE,NaN,18,18,-0.664416,2.945847,0.834296,0.165704,-0.081149,0.081149
1,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,461,(7:30) J.Goff pass short right to J.Reynolds t...,1,...,COVER_6_ZONE,NaN,21,21,1.926131,1.345633,0.544618,0.455382,-0.029415,0.029415
2,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1940,(:09) (Shotgun) J.Goff pass incomplete deep ri...,2,...,COVER_2_ZONE,NaN,0,0,0.281891,-0.081964,0.771994,0.228006,0.000791,-0.000791
3,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1711,"(:45) (No Huddle, Shotgun) P.Mahomes pass deep...",2,...,COVER_2_ZONE,NaN,26,26,3.452352,2.342947,0.663187,0.336813,0.041843,-0.041843
4,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1588,(1:54) (Shotgun) P.Mahomes pass incomplete dee...,2,...,COVER_4_ZONE,NaN,0,0,1.921525,-0.324035,0.615035,0.384965,0.000061,-0.000061


## Transform Parameters 

In [54]:
# function to transform the x coordinate 
def calc_new_x(y, flip_y = False):
    xnew = 53.3 - y 

    # flip if needed 
    if flip_y:
        xnew = 53.3 - xnew 

    return xnew 

# function to transform the y coordinate 
def calc_new_y(x, flip_y = False):
    ynew = x 

    # flip if needed 
    if flip_y:
        ynew = 120 - ynew 

    return ynew 

# function to transform the angle 
def calc_new_angle(angle, flip_y = False):
    anglenew = angle - 90

    # flip if needed 
    if flip_y:
        anglenew = (anglenew + 180) % 360 

    return anglenew  

## PlayDataSetup 

In [55]:
# class to setup the play data into keyframes and routes for visualization 
class PlayDataSetup:

    def __init__(self, dfp, setup_type = "input", flip_y = False):
        self.setup_type = setup_type 

        # columns to keep 
        cnames = ["nfl_id", "frame_id", "xc", "yc", "s"] 

        # sort by frame 
        dfp = dfp.sort_values(["nfl_id", "frame_id"]).reset_index(drop = True) 

        # transform the coordinates and angles based on the setup type 
        dfp["xc"] = calc_new_x(dfp["y"], flip_y = flip_y) 
        dfp["yc"] = calc_new_y(dfp["x"], flip_y = flip_y) 
        if setup_type == "input":
            dfp["angle"] = calc_new_angle(dfp["o"], flip_y = flip_y) 
            cnames += ["angle"] 
        
        else:
            dfp["xc_last"] = dfp.groupby("nfl_id")["xc"].shift(1) 
            dfp["yc_last"] = dfp.groupby("nfl_id")["yc"].shift(1) 
            dfp["s"] = np.sqrt((dfp["xc"] - dfp["xc_last"])**2 + (dfp["yc"] - dfp["yc_last"])**2) / 10 
            dfp["s"] = dfp.groupby("nfl_id")["s"].bfill() 
            dfp = dfp.drop(columns = ["xc_last", "yc_last"]) 

        # # transform the coordinates to match our field orientation 
        # dfp["xc"] = 53.3 - dfp["y"] 
        # dfp["yc"] = dfp["x"] 

        # # flip the y-coordinates if needed 
        # if flip_y:
        #     dfp["xc"] = 53.3 - dfp["xc"] 
        #     dfp["yc"] = 120 - dfp["yc"] 
        
        # # add the angle if needed 
        # cnames = ["nfl_id", "frame_id", "xc", "yc"] 
        # if setup_type == "input":
        #     dfp["angle"] = dfp["o"] - 90 
        #     cnames += ["angle"] 

        #     # adjust the angle if needed 
        #     if flip_y: 
        #         dfp["angle"] = (dfp["angle"] + 180) % 360 
        
        # add as an attribute 
        self.dfp = dfp[cnames] 
    
    # function to get data for a specific player 
    def get_player_data(self, player_id):
        return self.dfp.loc[self.dfp["nfl_id"] == player_id].reset_index(drop = True) 

    # subset frames by a given min/max 
    def subset_player_frames(self, player_id, min_frame = None, max_frame = None):

        # filter for the given player
        df_subset = self.get_player_data(player_id) 

        # subset by min (if given)
        if min_frame is not None:
            df_subset = df_subset.loc[df_subset["frame_id"] >= min_frame]
        
        # subset by max (if given) 
        if max_frame is not None:
            df_subset = df_subset.loc[df_subset["frame_id"] <= max_frame]
        
        return df_subset.reset_index(drop = True) 
    
    # create keyframes for a given player 
    def create_player_keyframes(self, player_id, start_time = 0, fps = 10, min_frame = None, max_frame = None):

        # get the player data 
        df_player = self.subset_player_frames(player_id, min_frame = min_frame, max_frame = max_frame) 

        # get the first frame 
        frame_start = df_player["frame_id"].min() 

        # iterate through each frame and create keyframes 
        keyframes = [] 
        for i, row in df_player.iterrows():
            kf_new = {
                "xc": row["xc"], 
                "yc": row["yc"] 
            }

            # add the angle if needed 
            if self.setup_type == "input":
                kf_new["angle"] = row["angle"] 

            # add the keyframes 
            keyframes.append({
                "time": start_time + ((row["frame_id"] - frame_start) / fps), 
                "values": kf_new 
            }) 
        
        return keyframes 
    
    # create the route definition for a given player 
    def create_route_definition(self, player_id, start_time = 0, fps = 10, min_frame = None, max_frame = None):

        # get the player data 
        df_player = self.subset_player_frames(player_id, min_frame = min_frame, max_frame = max_frame) 

        # create the route definition 
        route_def = {
            "start_time": start_time, 
            "fps": fps, 
            "path": {
                "frame_id": df_player["frame_id"].tolist(), 
                "xc": df_player["xc"].tolist(), 
                "yc": df_player["yc"].tolist() 
            }
        } 

        return route_def 
    
    # create keyframes for the speed of a given player 
    def create_speed_keyframes(self, player_id, start_time = 0, fps = 10, min_frame = None, max_frame = None, x_off = 0, y_off = 0):

        # get the player data 
        df_player = self.subset_player_frames(player_id, min_frame = min_frame, max_frame = max_frame) 

        # get the first frame 
        frame_start = df_player["frame_id"].min() 

        # iterate through each frame and create keyframes 
        keyframes = [] 
        for i, row in df_player.iterrows():
            kf_new = {
                "xc": row["xc"] + x_off, 
                "yc": row["yc"] + y_off, 
                "value": row["s"] 
            } 

            # add the keyframes 
            keyframes.append({
                "time": start_time + ((row["frame_id"] - frame_start) / fps), 
                "values": kf_new 
            }) 
        
        return keyframes 

## PlayDataLoader 

In [56]:
# class to load in and transform the data for a given play 
class PlayDataLoader: 

    def __init__(self, game_id, play_id, df_meta): 
        self.game_id = game_id 
        self.play_id = play_id 

        # filter the metadata for the given play 
        self.play_meta = df_meta.loc[
            (df_meta["game_id"] == game_id) & 
            (df_meta["play_id"] == play_id)
        ].reset_index(drop = True).to_dict(orient = "records")[0] 

        # load in the input and output data 
        df_input = pd.read_csv(f"{folder_path}//train//input_2023_w{self.play_meta['week']:02}.csv") 
        df_output = pd.read_csv(f"{folder_path}//train//output_2023_w{self.play_meta['week']:02}.csv") 

        # filter for the given play 
        self.df_input = df_input.loc[
            (df_input["game_id"] == game_id) & 
            (df_input["play_id"] == play_id)
        ].reset_index(drop = True) 
        self.df_output = df_output.loc[
            (df_output["game_id"] == game_id) & 
            (df_output["play_id"] == play_id) 
        ].reset_index(drop = True) 

        # get the frame min/max values 
        self.input_min = self.df_input["frame_id"].min() 
        self.input_max = self.df_input["frame_id"].max() 
        self.output_min = self.df_output["frame_id"].min() 
        self.output_max = self.df_output["frame_id"].max() 

        # get the play direction 
        avg_start = self.df_input[self.df_input["frame_id"] == self.input_min]["x"].mean() 
        avg_end = self.df_input[self.df_input["frame_id"] == self.input_max]["x"].mean() 
        self.play_direction = "right" if avg_end > avg_start else "left" 
        self.flip_y = (self.play_direction == "left") 

        # setup the input data for visualization 
        self.pds_input = PlayDataSetup(
            dfp = self.df_input,
            setup_type = "input", 
            flip_y = self.flip_y 
        ) 

        # setup the output data for visualization 
        self.pds_output = PlayDataSetup(
            dfp = self.df_output,
            setup_type = "output", 
            flip_y = self.flip_y 
        ) 

        # get the final ball position 
        final_frame = self.df_input.sort_values("frame_id").iloc[-1] 
        self.ball_land = [
            calc_new_x(final_frame["ball_land_y"], self.flip_y), 
            calc_new_y(final_frame["ball_land_x"], self.flip_y) 
        ] 

        # get the player positions 
        df_positions = self.df_input[["nfl_id", "player_position"]].drop_duplicates() 
        self.positions = {} 
        for i, row in df_positions.iterrows():
            self.positions[row['nfl_id']] = row['player_position'] 

    # get unique players 
    def get_unique_players(self): 
        return self.df_input[["nfl_id", "player_side", "player_to_predict"]].drop_duplicates() 
    
    # calculate the total time of the play (before the pass) 
    def calc_play_time(self, fps = 10):
        return {
            "before_pass": (self.df_input["frame_id"].nunique()) / fps, 
            "during_pass": (self.df_output["frame_id"].nunique()) / fps 
        }
    
    # calculate the window bounds of the play 
    def calc_play_window(self, padding = 5): 
        window = [
            max(min(self.df_input["x"].min(), self.df_output["x"].min()) - padding, 0), 
            min(max(self.df_input["x"].max(), self.df_output["x"].max()) + padding, 120) 
        ]

        # flip the window if needed 
        if self.play_direction == "left":
            window = [120 - window[1], 120 - window[0]] 

        return window

# sample usage 
pdl = PlayDataLoader(
    game_id = 2023121700, 
    play_id = 2553, 
    df_meta = df_meta 
) 
pdl.play_meta  

{'game_id': 2023121700,
 'season': 2023,
 'week': 15,
 'game_date': '12/17/2023',
 'game_time_eastern': '13:00:00',
 'home_team_abbr': 'GB',
 'visitor_team_abbr': 'TB',
 'play_id': 2553,
 'play_description': '(4:26) (Shotgun) J.Love pass deep right to J.Reed for 17 yards, TOUCHDOWN.',
 'quarter': 3,
 'game_clock': '04:26',
 'down': 3,
 'yards_to_go': 14,
 'possession_team': 'GB',
 'defensive_team': 'TB',
 'yardline_side': 'TB',
 'yardline_number': 17,
 'pre_snap_home_score': 10,
 'pre_snap_visitor_score': 20,
 'play_nullified_by_penalty': 'N',
 'pass_result': 'C',
 'pass_length': 17,
 'offense_formation': 'EMPTY',
 'receiver_alignment': '3x2',
 'route_of_targeted_receiver': 'CORNER',
 'play_action': False,
 'dropback_type': 'SCRAMBLE',
 'dropback_distance': 5.8899998664856,
 'pass_location_type': 'OUTSIDE_RIGHT',
 'defenders_in_the_box': 6,
 'team_coverage_man_zone': 'ZONE_COVERAGE',
 'team_coverage_type': 'COVER_4_ZONE',
 'penalty_yards': nan,
 'pre_penalty_yards_gained': 17,
 'yards_

## setup_play_showcase 

In [57]:
# function to setup an animation for a given play 
def setup_play_showcase(game_id, play_id, df_meta): 

    # import the play data 
    pdl = PlayDataLoader(
        game_id = game_id, 
        play_id = play_id, 
        df_meta = df_meta
    ) 

    # get some info about the play 
    df_players = pdl.get_unique_players() 
    pbounds = pdl.calc_play_window() 

    # create the play animation with the field 
    pa = PlayAnimation(ystart = pbounds[0], yend = pbounds[1]) 
    pa.add_object(FieldObject()) 

    # pause time right before the pass 
    pause_time = 1 

    # add the final ball landing position 
    pa.add_object(FootballObject(id = "ball")) 
    pa.add_keyframes({
        "ball": [
            {
                "time": pdl.calc_play_time()["before_pass"] + pause_time + pdl.calc_play_time()["during_pass"], 
                "values": {
                    "xc": pdl.ball_land[0], 
                    "yc": pdl.ball_land[1], 
                    "angle": 0 
                }
            } 
        ] 
    }) 

    # add some players 
    for i, row in df_players.iterrows():
        player_id = row["nfl_id"] 

        # set the color for the player 
        if row["player_side"] == "Defense": 
            pcolor = "#FF0000" 
        elif row["player_to_predict"]:
            pcolor = "#9999FF" 
        else:
            pcolor = "#0000FF"

        # add the player objects 
        pa.add_object(PlayerObject(id = f"{player_id}", pcolor = pcolor)) 
        pa.add_object(MetricObject(id = f"{player_id}_speed", suffix = " yd/s"))

        # add the input keyframes and routes 
        keyframes = pdl.pds_input.create_player_keyframes(player_id = player_id) 
        pa.add_keyframes({f"{player_id}": keyframes}) 
        speed_keyframes = pdl.pds_input.create_speed_keyframes(
            player_id = player_id, 
            x_off = 0, 
            y_off = -2 
        ) 
        pa.add_keyframes({f"{player_id}_speed": speed_keyframes}) 
        route_def = pdl.pds_input.create_route_definition(player_id = player_id) 
        pa.add_object(RouteObject(
            id = f"{player_id}_route_before", 
            routes = [route_def], 
            rcolor = pcolor
        )) 

        # add the ball object for the quarterback 
        if pdl.positions[player_id] == "QB": 
            keyframes_ball = pdl.pds_input.create_player_keyframes(
                player_id = player_id, 
                min_frame = pdl.input_min, 
                max_frame = pdl.input_max
            ) 
            pa.add_keyframes({"ball": keyframes_ball}) 

        # add the output keyframes and routes 
        start_time = pdl.calc_play_time()["before_pass"] + pause_time
        keyframes = pdl.pds_output.create_player_keyframes(player_id = player_id, start_time = start_time) 
        pa.add_keyframes({f"{player_id}": keyframes}) 
        speed_keyframes = pdl.pds_output.create_speed_keyframes(
            player_id = player_id, 
            x_off = 0, 
            y_off = -2, 
            start_time = start_time 
        ) 
        pa.add_keyframes({f"{player_id}_speed": speed_keyframes}) 
        route_def = pdl.pds_output.create_route_definition(player_id = player_id, start_time = start_time) 
        pa.add_object(RouteObject(
            id = f"{player_id}_route_during", 
            routes = [route_def], 
            rcolor = "black"
        )) 

    
    return pa, pdl 

# create the play 
pa, pdl = setup_play_showcase(
    game_id = 2023121700, 
    play_id = 2553, 
    df_meta = df_meta 
) 

# test the image at a given time 
pa.save_image(t = 9) 

# # create and save the animation 
# pa.create_animation(
#     end_time = 9, 
#     fps = 12, 
#     save_file = "test_animation.mp4"
# ) 

# close out the plot 
plt.close() 

## Animate Example 

In [58]:
# create the play 
pa, pdl = setup_play_showcase(
    game_id = 2023102904, 
    play_id = 838, 
    df_meta = df_meta 
) 

# calculate the end time for the animation 
ptimes = pdl.calc_play_time() 
end_time = ptimes["before_pass"] + 1 + ptimes["during_pass"] + 1

# create and save the animation 
pa.create_animation(
    end_time = end_time, 
    fps = 12, 
    save_file = f"{results_path}//play_example.mp4"
) 

# close out the plot 
plt.close() 

  Creating frame 1 / 90...
  Creating frame 21 / 90...
  Creating frame 41 / 90...
  Creating frame 61 / 90...
  Creating frame 81 / 90...


# Save Screenshots 

In [59]:
def save_play_screenshots(plays, df_meta): 

    # dummy variables 
    html_items = [] 

    # iterate through each play 
    for play in plays:
        game_id = play[0] 
        play_id = play[1] 
        print(f"Processing game {game_id} play {play_id}...") 

        try:

            # create the play 
            pa, pdl = setup_play_showcase(
                game_id = game_id, 
                play_id = play_id, 
                df_meta = df_meta 
            ) 

            # get the end time 
            throw_time = pdl.calc_play_time()["before_pass"]
            end_time = throw_time + 1 + pdl.calc_play_time()["during_pass"] 

            # save images of the start and end times 
            pa.save_image(t = 0, save_file = f"{results_path}//Images//game_{game_id}_play_{play_id}_start.png") 
            pa.save_image(t = throw_time, save_file = f"{results_path}//Images//game_{game_id}_play_{play_id}_throw.png") 
            pa.save_image(t = end_time, save_file = f"{results_path}//Images//game_{game_id}_play_{play_id}_end.png") 

            # close out the plot 
            plt.close() 

            # function to make a number into a string with the appropriate suffix (i.e. 1st, 2nd, 3rd, etc.) 
            def number_to_suffix(n):
                if 10 <= n % 100 <= 20:
                    suffix = 'th'
                else:
                    suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
                return str(n) + suffix 

            # add to the html report 
            html_items.append(f'''
        <div class="play-showcase">
            <h2 class="play-title">Game: {game_id} | Play: {play_id}</h2> 
            <div class="play-summary">Week {pdl.play_meta["week"]} ({pdl.play_meta["game_date"]}): {pdl.play_meta["visitor_team_abbr"]} at {pdl.play_meta["home_team_abbr"]}</div> 
            <div class="play-summary">{number_to_suffix(pdl.play_meta["quarter"])} Quarter with {pdl.play_meta["game_clock"]} remaining - {number_to_suffix(pdl.play_meta["down"])} and {pdl.play_meta["yards_to_go"]} on {pdl.play_meta["yardline_side"]} {pdl.play_meta["yardline_number"]}</div> 
            <div class="play-summary">{pdl.play_meta["play_description"]}</div>
            <div class="play-summary">{pdl.play_meta["route_of_targeted_receiver"]} route against {pdl.play_meta["team_coverage_type"]}</div> 
            <div class="container" style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; max-width: 1640px; margin: 0 auto;">
                <div>
                    <img src="Images/game_{game_id}_play_{play_id}_throw.png" style="width:800px;">
                </div> 
                <div>
                    <img src="Images/game_{game_id}_play_{play_id}_end.png" style="width:800px;">
                </div> 
                <div>
                    <img src="Images/game_{game_id}_play_{play_id}_start.png" style="width:800px;">
                </div>
            </div>
        </div> ''') 
    
        except Exception as e:
            print(f"  Error processing game {game_id} play {play_id}: {e}") 
            continue 
    
    # combine the html code 
    html_code = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Output Test</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #FFFFFF;
        }} 
        .play-showcase {{
            border: 3px solid #000000;
            border-radius: 8px;
            padding: 20px;
            margin: 20px auto;
            max-width: 1860px;
            background-color: #f8f9fa;
        }}
        .play-title {{
            text-align: center;
            font-size: 24px;
            font-weight: bold;
            margin-bottom: 20px;
            color: #333;
        }}
        .play-summary {{
            font-size: 18px;
            margin-bottom: 15px;
            margin-left: 50px; 
            color: #555;
        }}
    </style>
</head>
<body> {''.join(html_items)}
</body>
</html>
''' 
    
    # save the html file 
    with open(f"{results_path}/play_screenshots.html", "w") as f:
        f.write(html_code) 

# # list of plays to process 
# plays = [
#     [2023121700, 2553], 
#     [2023121700, 1954], 
#     [2023121700, 1648], 
#     [2023121700, 1623], 
#     [2023121700, 1113], 
#     [2023121700, 2039], 
#     [2023121700, 2382], 
#     [2023121700, 1341], 
#     [2023121700, 2814], 
#     [2023121700, 3575], 
#     [2023121700, 3598]
# ] 

# # save the play screenshots 
# save_play_screenshots(plays, df_meta) 

## More Examples 

In [60]:
# list of plays to process 
plays = [
    # [2023102600, 3729], 
    # [2023120304, 1905], 
    # [2023100107, 1968], 
    # [2023102205, 1057], 
    # [2023102600, 3953], 
    # [2023111206, 2475], 
    # [2023111908, 1369], 
    # [2023091010, 3198], 
    # [2023091010, 4426], 
    # [2023100804, 2167], 
    # [2023101507, 1830], 
    # [2023102904, 838], 
    [2023120700, 3817], 
    [2023121801, 1627], 
    [2023120400, 2958], 
    [2023092100, 353], 
    # [2023102210, 1994], 
    # [2023120306, 1480], 
    # [2023120306, 388], 
] 

# save the play screenshots 
save_play_screenshots(plays, df_meta) 

Processing game 2023120700 play 3817...
Processing game 2023121801 play 1627...
Processing game 2023120400 play 2958...
Processing game 2023092100 play 353...


# Visualize Speed 

In [ ]:
def visualize_speed(game_id, play_id, player_ids):

    # load the play 
    pa, pdl = setup_play_showcase(
        game_id = game_id, 
        play_id = play_id, 
        df_meta = df_meta 
    ) 

    # create the figure 
    fig = go.Figure() 

    # add each player's speed 
    for player_id in player_ids: 

        # get the player dataframes 
        df1 = pdl.pds_input.subset_player_frames(player_id = player_id) 
        df2 = pdl.pds_output.subset_player_frames(player_id = player_id) 

        # combine the two dataframes 

        # calculate the speed values manually 
        input_frames = df1["frame_id"].max() 
        df2["frame_id"] = df2["frame_id"] + input_frames 
        df = pd.concat([df1, df2], ignore_index = True).sort_values("frame_id").reset_index(drop = True) 
        df["xc_last"] = df["xc"].shift(1) 
        df["yc_last"] = df["yc"].shift(1) 
        df["s"] = np.sqrt((df["xc"] - df["xc_last"])**2 + (df["yc"] - df["yc_last"])**2) * 10 

        # convert the speed values to mph 
        conversion_factor = 3600 / 1760 
        df["s"] = df["s"] * conversion_factor 
        # add trace for auto-calculated speed 
        fig.add_trace(go.Scatter(
            x = df["frame_id"] / 10, 
            y = df["s"], 
            mode='lines+markers', 
            name = player_id 
        )) 

    # style the figure 
    fig.update_layout(
        title = 'Speed',
        xaxis_title = 'Time (seconds)',
        yaxis_title = 'Speed (mph)',
        template='plotly_white'
    ) 

    # show the figure 
    fig.show() 

## Example 1 

In [62]:
visualize_speed(
    game_id = 2023120700, 
    play_id = 3817, 
    player_ids = [43700, 47849] 
)

## Example 2 

In [63]:
visualize_speed(
    game_id = 2023121801, 
    play_id = 1627, 
    player_ids = [55970] 
)

## Example 3 

In [64]:
visualize_speed(
    game_id = 2023120400, 
    play_id = 2958, 
    player_ids = [55925, 44849] 
)

## Example 4 

In [68]:
visualize_speed(
    game_id = 2023092100, 
    play_id = 353, 
    player_ids = [55888] 
)

# Test Calculations 

In [65]:
# load the play 
pa, pdl = setup_play_showcase(
    game_id = 2023102904, 
    play_id = 838, 
    df_meta = df_meta 
) 

In [66]:
player_id = 43454 

# get the player dataframes 
df1 = pdl.pds_input.subset_player_frames(player_id = player_id) 
df2 = pdl.pds_output.subset_player_frames(player_id = player_id) 

# calculate the speed values manually 
input_frames = df1["frame_id"].max() 
df2["frame_id"] = df2["frame_id"] + input_frames 
df = pd.concat([df1, df2], ignore_index = True).sort_values("frame_id").reset_index(drop = True) 
df["xc_last"] = df["xc"].shift(1) 
df["yc_last"] = df["yc"].shift(1) 
df["s_new"] = np.sqrt((df["xc"] - df["xc_last"])**2 + (df["yc"] - df["yc_last"])**2) * 10 

# convert the speed values to mph 
conversion_factor = 3600 / 1760 
df1["s"] = df1["s"] * conversion_factor 
df["s_new"] = df["s_new"] * conversion_factor 

# showcase the data 
df.head()  

,nfl_id,frame_id,xc,yc,s,angle,xc_last,yc_last,s_new
0,43454,1,16.75,66.88,0.07,350.56,NaN,NaN,NaN
1,43454,2,16.74,66.91,0.42,349.79,16.75,66.88,0.646830
2,43454,3,16.74,66.99,0.94,345.98,16.74,66.91,1.636364
3,43454,4,16.73,67.15,1.82,344.17,16.74,66.99,3.279113
4,43454,5,16.70,67.38,2.63,344.17,16.73,67.15,4.744396


In [67]:


fig = go.Figure()

# add trace for auto-calculated speed 
fig.add_trace(go.Scatter(
    x = df1["frame_id"] / 10, 
    y = df1["s"], 
    mode='lines+markers' 
)) 

# add trace for manually-calculated speed 
fig.add_trace(go.Scatter(
    x = df["frame_id"] / 10, 
    y = df["s_new"], 
    mode='lines+markers' 
)) 

# style the figure 
fig.update_layout(
    title = 'Speed',
    xaxis_title = 'Time (seconds)',
    yaxis_title = 'Distance to Ball (yards)',
    template='plotly_white'
) 

# show the figure 
fig.show() 